In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split as tts
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE 
from imblearn.pipeline import Pipeline # doctest: +NORMALIZE_WHITESPACE

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv('/content/drive/My Drive/train.csv')
rslt_df = df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)]
rslt_df2 = df[(df['toxic'] == 1) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)]
new1 = rslt_df[['id', 'comment_text', 'toxic']].iloc[:23891].copy() 
new2 = rslt_df2[['id', 'comment_text', 'toxic']].iloc[:946].copy()
new = pd.concat([new1, new2], ignore_index=True)

In [0]:
new.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=5)
Xv = vectorizer.fit(new['comment_text'])
import pickle
# # save the model to disk
# filename = 'tfidf1.pkl'
# pickle.dump(vectorizer, open(filename, 'wb'))

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new["comment_text"], new['toxic'], test_size=0.33)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=5)
X1 = vectorizer.transform(X_train)
X_test1= vectorizer.transform(X_test)

In [0]:
# import pickle
# # save the model to disk
# filename = 'tfidf.pkl'
# pickle.dump(vectorizer, open(filename, 'wb'))

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizerh = HashingVectorizer(n_features=2**23)
X = vectorizerh.fit_transform(X_train)
X_test= vectorizerh.transform(X_test)

In [0]:
# import pickle
# # save the model to disk
# filename = 'hashing.pkl'
# pickle.dump(vectorizer, open(filename, 'wb'))

In [12]:
print('Original dataset shape %s' % Counter(y_train))

Original dataset shape Counter({0: 16000, 1: 640})


In [13]:
sm = SVMSMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Resampled dataset shape Counter({0: 16000, 1: 16000})


In [0]:
from sklearn.svm import SVC
f = SVC(gamma='auto')
y_p = f.fit(X_res, y_res).predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_p)
print('Accuracy: %f' % accuracy)

Accuracy: 0.935830


In [0]:
# import pickle
# # save the model to disk
# filename = 'finalized_model3.pkl'
# pickle.dump(f, open(filename, 'wb'))

In [17]:
import numpy as np

z=1.96
interval = z * np.sqrt( (0.96 * (1 - 0.96)) / y_test.shape[0])
interval

0.00424223005015792

In [18]:
sm = SMOTE(random_state=12)
x_train_res, y_train_res = sm.fit_sample(X1, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
from sklearn.svm import SVC
f1 = SVC(gamma='auto')
y_p2 = f1.fit(x_train_res, y_train_res).predict(X_test1)

In [20]:
type(f)

sklearn.svm._classes.SVC

In [21]:
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_p2)
print('Accuracy: %f' % accuracy)

Accuracy: 0.952910


In [0]:
# import pickle
# # save the model to disk
# filename = 'final.pkl'
# pickle.dump(f1, open(filename, 'wb'))

In [23]:
import numpy as np

z=1.96
interval = z * np.sqrt( (0.959 * (1 - 0.959)) / y_test.shape[0])
interval

0.004292693054894032

LOGISTIC REGRESSION

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
m = clf.fit(x_train_res, y_train_res).predict(X_test1)

In [25]:
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, m)
print('Accuracy: %f' % accuracy)

Accuracy: 0.950104


In [26]:
import numpy as np

z=1.96
interval = z * np.sqrt( (0.95 * (1 - 0.95)) / y_test.shape[0])
interval

0.004718189813401545

In [0]:
# import pickle
# # save the model to disk
# filename = 'final_lr.pkl'
# pickle.dump(clf, open(filename, 'wb'))

RandomForest

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [0]:
# build normal model
# pipeline = make_pipeline(classifier(random_state=42))
model = classifier.fit(x_train_res, y_train_res)
prediction = model.predict(X_test1)

In [30]:
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, prediction)
print('Accuracy: %f' % accuracy)

Accuracy: 0.963035


In [31]:
import numpy as np

z=1.96
interval = z * np.sqrt( (0.96 * (1 - 0.96)) / y_test.shape[0])
interval

0.00424223005015792

Multinomial NB

In [0]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
y_pred = clf.fit(x_train_res, y_train_res).predict(X_test1)

In [33]:
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)

Accuracy: 0.897889


In [34]:
import numpy as np

z=1.96
interval = z * np.sqrt( (0.89 * (1 - 0.89)) / y_test.shape[0])
interval

0.006773606284747332

In [0]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid','linear']}

In [0]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(x_train_res, y_train_res)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....................... C=0.1, gamma=1, kernel=rbf, total= 6.2min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.2min remaining:    0.0s


[CV] ....................... C=0.1, gamma=1, kernel=rbf, total= 6.3min
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total= 6.4min
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total= 6.4min
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total= 6.2min
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=13.0min
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=13.0min
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=13.1min
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] .

In [0]:
print(grid.best_estimator_)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix  
grid_predictions = grid.predict(X_test1)
print(confusion_matrix(y_test,grid_predictions))
print(classification_report(y_test,grid_predictions))